# Data Visualisation of COVID-19 in the UK using Sqlite, Pandas and Seaborn Libraries 
## Part 2: Extract and Load the Data

This is the second notebook in a 3-part series which explores the UK Gov's COVID-19 dashboard data which is publically available for download via a REST API from gov.uk.  We make use of a Sqlite3 database to query this data using SQL and import aggregations into Pandas data frames.  We then use the Seaborn library to visualise the results.

In this notebook (Part 2) - we first extract the data using a REST API based on the GOV.UK COVID-19 SDK and then insert the data into empty tables created in Part 1.  Part 3 will then query and visualise the data.

The data used in this notebook is publically available and more information can be found here:
https://coronavirus.data.gov.uk/details/about-data

### Configuration and Setup

The main library we will use to perform the REST API calls to get the GOV.UK COVID-19 data is the SDK published by GOV.UK.  This will handle the various calls, but also pagination as well and populate the data into a Pandas data-frame.

Install the library using `pip install uk-covid19`

For more information on the SDK, see: https://pypi.org/project/uk-covid19/

In [1]:
from uk_covid19 import Cov19API  # Use the UK COVID-19 GOV.UK SDK
import pandas as pd              # Pandas library for Dataframes
import time                      # Need the Sleep function for API calls
import csv                       # For CSV generation
from datetime import datetime    # Datetime functionality
import re                        # Regular Expression for the snake_case function

### Camel Case to Snake Case Function for Column Names

We're going to use a naming standard for the SQL tables based on Snake Case (lowercase and underscore only).  One reason we might do this is because it's more compliant with various database systems such as HiveQL, Amazon Athena, etc.

Unfortunately the column naming used by the GOV.UK API is based on Camel Case (mixed case with no spaces).  So the following function will be used to convert the source column names into snake case which will make importing the data easier.

In [2]:
def camel_to_snake(name: str) -> str:
    """
    Description: Convert any string to snake case (lower case and _ for spacing)
    Args:        name: input string to convert
    Returns:     input string converted to snake case 
    """   
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    name = re.sub('[,.]', '_', name)   
    name = re.sub('[+*&%=()?<>!@#$/\\\\]', '', name)  

    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower().replace(" ","_").replace("-","_").replace("__","_").replace("__","_").replace("deaths28","deaths_28")       

### Handling Metrics and Area Types in the REST API

If you read the documentation provided at https://coronavirus.data.gov.uk/details/download and https://coronavirus.data.gov.uk/details/developers-guide - you'll see that only a maximum of 5 metrics can be requested via the API in addition to the standard (primary key) metrics: Area Type, Name, Code and Date.

This means we have to make multiple API requests for groups of metrics since we can't request all in one go.  Additionally the metrics available differ depending on the Area Type you are requesting.

The following provides a configuration in the form of a dictionary object which define the metrics available for each area type.  In addition the metrics are broken down into groups of no more than 5 metrics and this will allow us to iterate in a more generic function.  The default metrics are also provided in a separate list object.

This notebook is going to retrieve data for all area types, and for most of the metrics available.  However, this is forever changing so as more metrics become available we simply add them to one or more separate metric groups for the appropriate area type.

In [3]:
default_metrics = ["areaType",
                   "areaName",
                   "areaCode",
                   "date"]

metrics_by_area_type = {
    "overview" : [
        ["newCasesByPublishDate","cumCasesByPublishDate","cumCasesByPublishDateRate","newCasesBySpecimenDate","cumCasesBySpecimenDate"],
        ["cumCasesBySpecimenDateRate","newPillarOneTestsByPublishDate","cumPillarOneTestsByPublishDate"],
        ["newPillarTwoTestsByPublishDate","cumPillarTwoTestsByPublishDate","newPillarThreeTestsByPublishDate","cumPillarThreeTestsByPublishDate","newPillarFourTestsByPublishDate"],
        ["cumPillarFourTestsByPublishDate","newAdmissions","cumAdmissions","cumTestsByPublishDate"],
        ["newTestsByPublishDate","covidOccupiedMVBeds","hospitalCases","plannedCapacityByPublishDate","newDeaths28DaysByPublishDate","cumDeaths28DaysByPublishDate"],
        ["cumDeaths28DaysByPublishDateRate","newDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDateRate"]
    ],
    "nation" : [
        ["newCasesByPublishDate","cumCasesByPublishDate","cumCasesByPublishDateRate","newCasesBySpecimenDate","cumCasesBySpecimenDate"],
        ["cumCasesBySpecimenDateRate","maleCases","femaleCases","newPillarOneTestsByPublishDate","cumPillarOneTestsByPublishDate"],
        ["newPillarTwoTestsByPublishDate","cumPillarTwoTestsByPublishDate","newPillarThreeTestsByPublishDate","cumPillarThreeTestsByPublishDate"],
        ["newAdmissions","cumAdmissions","cumAdmissionsByAge","cumTestsByPublishDate"],
        ["newTestsByPublishDate","covidOccupiedMVBeds","hospitalCases","newDeaths28DaysByPublishDate","cumDeaths28DaysByPublishDate"],
        ["cumDeaths28DaysByPublishDateRate","newDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDateRate"]
    ],
    "region" : [
        ["newCasesByPublishDate","cumCasesByPublishDate","cumCasesByPublishDateRate","newCasesBySpecimenDate","cumCasesBySpecimenDate"],
        ["cumCasesBySpecimenDateRate","maleCases","femaleCases","newDeaths28DaysByPublishDate","cumDeaths28DaysByPublishDate"],
        ["cumDeaths28DaysByPublishDateRate","newDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDateRate"]
    ],  
    "nhsRegion" : [
        ["newAdmissions","cumAdmissions","cumAdmissionsByAge","covidOccupiedMVBeds","hospitalCases"]
    ],  
    "utla" : [
        ["newCasesByPublishDate","cumCasesByPublishDate","cumCasesByPublishDateRate","newCasesBySpecimenDate","cumCasesBySpecimenDate"],
        ["cumCasesBySpecimenDateRate","newDeaths28DaysByPublishDate","cumDeaths28DaysByPublishDate"],
        ["cumDeaths28DaysByPublishDateRate","newDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDateRate"]
    ],
    "ltla" : [
        ["newCasesByPublishDate","cumCasesByPublishDate","cumCasesByPublishDateRate","newCasesBySpecimenDate","cumCasesBySpecimenDate"],
        ["cumCasesBySpecimenDateRate","newDeaths28DaysByPublishDate","cumDeaths28DaysByPublishDate"],
        ["cumDeaths28DaysByPublishDateRate","newDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDate","cumDeaths28DaysByDeathDateRate"]
    ]    
}

### Extract the Data as CSV by iterating through the metric groups

Now we simply iterate through each area type which provides a list of metric groups.  Then we iterate through each metric group (no more than 5 metrics) and combined with the mandatory default metrics we use the SDK to make a REST API call and grab the data into a Pandas dataframe.

Unfortunately there is rate limiting implemented for this API as documentated in the fair usage policy: https://coronavirus.data.gov.uk/details/download.  As such we use `time.sleep(1)` to pause for one second between each API call.  So far this has provded sufficient to abide by the fair usage policy.

As we iterate through each metric group, we combine all the data together using an outer join merge based on the default metric columns.

We then convert the column names to snake case, convert the `date` column to datetime, and finally write the data out to a CSV (so one CSV per area type is generated).

**So why CSV and not load straight to the database?** 

It's common in most ETL/ELT processing systems to initially stage the data before loading into a database - typically to a data-lake of some sort.  Here we simply use the file system and stage the data as CSV.  This means if we have any errors loading the data into the database tables, we have the data in a ready to use format to investigate the issues further.  

**NOTE:** The following code will take several minutes to complete.

In [4]:
# iterate through each area type (this provides a list of metric groups)
# key = areaType, value = list of metric groups (or a list of lists of metrics)
for key, value in metrics_by_area_type.items():
    
    filters = [f"areaType={key}"]
    
    df_merged = pd.DataFrame()
    
    # iterate through each metric group (at most 5 metrics)
    metric_group_index = 1
    for metric_group in value:
        # we create a dictionary containing the metrics which must also include the default metrics
        structure = {}
        structure_d = {k: k for k in default_metrics}
        structure_m = {k: k for k in metric_group}
        structure.update(structure_d)
        structure.update(structure_m)

        # call the API using the SDK - requires just the Area Type and Metrics requested 
        api = Cov19API(
            filters=filters,
            structure=structure
        ) 
        
        # grab data as a Pandas dataframe and merge with any previously requested metrics
        df = api.get_dataframe() 
        print(f"{datetime.now().strftime('%H:%M:%S')} : Area Type: {key} - Metric Group {str(metric_group_index)} - Num records: {len(df)}")
        if df_merged.empty:
            df_merged = df
        else:
            df_merged = df_merged.merge(df, on=default_metrics, how="outer")
            
        # pause for a second to avoid breaching the fair usage policy
        time.sleep(1.0)
        metric_group_index += 1
     
    # all the metrics have now been retrieved and merged so convert columns to snake_case and write out the area type's CSV data
    df_merged.rename(columns=camel_to_snake, inplace=True)
    df_merged["date"]= pd.to_datetime(df_merged["date"])
    df_merged.to_csv(f"{key}.csv", index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

12:29:07 : Area Type: overview - Metric Group 1 - Num records: 565
12:29:11 : Area Type: overview - Metric Group 2 - Num records: 535
12:29:14 : Area Type: overview - Metric Group 3 - Num records: 503
12:29:18 : Area Type: overview - Metric Group 4 - Num records: 511
12:29:21 : Area Type: overview - Metric Group 5 - Num records: 529
12:29:25 : Area Type: overview - Metric Group 6 - Num records: 533
12:29:28 : Area Type: nation - Metric Group 1 - Num records: 2168
12:29:30 : Area Type: nation - Metric Group 2 - Num records: 2156
12:29:32 : Area Type: nation - Metric Group 3 - Num records: 1749
12:29:33 : Area Type: nation - Metric Group 4 - Num records: 2091
12:29:35 : Area Type: nation - Metric Group 5 - Num records: 2102
12:29:37 : Area Type: nation - Metric Group 6 - Num records: 2092
12:29:39 : Area Type: region - Metric Group 1 - Num records: 4899
12:29:43 : Area Type: region - Metric Group 2 - Num records: 4899
12:29:45 : Area Type: region - Metric Group 3 - Num records: 4744
12:2

### Supplementary Downloads

We need to download population data for the various geographies (such as UTLA) which are available from GOV.UK.  This will help provide standardised case and death rates between different areas.  

As was the case for the COVID-19 data, we snake case the column names.

In [8]:
df_populations = pd.read_csv(f"https://coronavirus.data.gov.uk/downloads/supplements/ONS-population_2021-08-05.csv")
df_populations.rename(columns=camel_to_snake, inplace=True)
df_populations.to_csv(f"populations.csv", index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

In [10]:
import sqlite3

sqlite_db_path = "c19.db"
conn = sqlite3.connect(sqlite_db_path)

In [11]:
pd.read_csv(f"nation.csv").to_sql("c19dashboard_uk__national_daily_metrics", conn, if_exists="append", index=False)
pd.read_csv(f"overview.csv").to_sql("c19dashboard_uk__summary_daily_metrics", conn, if_exists="append", index=False)
pd.read_csv(f"populations.csv").to_sql("reference_geography__age_gender_populations", conn, if_exists="append", index=False)

In [12]:
pd.read_sql("""
SELECT * FROM c19dashboard_uk__summary_daily_metrics
LIMIT 100
""",conn)

,area_type,area_name,area_code,date,new_cases_by_publish_date,cum_cases_by_publish_date,cum_cases_by_publish_date_rate,new_cases_by_specimen_date,cum_cases_by_specimen_date,cum_cases_by_specimen_date_rate,...,new_tests_by_publish_date,covid_occupied_mv_beds,hospital_cases,planned_capacity_by_publish_date,new_deaths_28_days_by_publish_date,cum_deaths_28_days_by_publish_date,cum_deaths_28_days_by_publish_date_rate,new_deaths_28_days_by_death_date,cum_deaths_28_days_by_death_date,cum_deaths_28_days_by_death_date_rate
0,overview,United Kingdom,K02000001,2021-08-16,28438,6295613,9385.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,26,130979,195.3,NaN,NaN,NaN
1,overview,United Kingdom,K02000001,2021-08-15,26750,6267437,9343.1,6860.0,6295613.0,9385.1,...,674509.0,NaN,NaN,842449.0,61,130953,195.2,8.0,130978.0,195.3
2,overview,United Kingdom,K02000001,2021-08-14,29520,6241011,9303.7,21971.0,6288753.0,9374.8,...,656190.0,NaN,NaN,842389.0,93,130894,195.1,22.0,130970.0,195.2
3,overview,United Kingdom,K02000001,2021-08-13,32700,6211868,9260.2,27632.0,6266782.0,9342.1,...,826362.0,NaN,NaN,832269.0,100,130801,195.0,35.0,130948.0,195.2
4,overview,United Kingdom,K02000001,2021-08-12,33074,6179506,9212.0,30709.0,6239150.0,9300.9,...,815286.0,882.0,5894.0,833019.0,94,130701,194.8,76.0,130913.0,195.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,overview,United Kingdom,K02000001,2021-05-13,2657,4444631,6625.7,2092.0,4442705.0,6622.9,...,1023529.0,129.0,1000.0,768238.0,11,127651,190.3,5.0,127777.0,190.5
96,overview,United Kingdom,K02000001,2021-05-12,2284,4441975,6621.8,2230.0,4440613.0,6619.8,...,1209469.0,136.0,1067.0,773238.0,11,127640,190.3,11.0,127772.0,190.5
97,overview,United Kingdom,K02000001,2021-05-11,2474,4439691,6618.4,2197.0,4438383.0,6616.4,...,713918.0,138.0,1105.0,768238.0,20,127629,190.3,14.0,127761.0,190.5
98,overview,United Kingdom,K02000001,2021-05-10,2357,4437217,6614.7,2652.0,4436186.0,6613.2,...,957267.0,149.0,1128.0,757980.0,4,127609,190.2,8.0,127747.0,190.4


In [13]:
pd.read_sql("""
SELECT * FROM reference_geography__age_gender_populations
LIMIT 100
""",conn)

,category,area_code,gender,age,population
0,AGE_ONLY,E06000001,ALL,00_04,5147
1,AGE_ONLY,E06000001,ALL,00_59,69434
2,AGE_ONLY,E06000001,ALL,05_09,5774
3,AGE_ONLY,E06000001,ALL,10_14,5982
4,AGE_ONLY,E06000001,ALL,15_19,5126
...,...,...,...,...,...
95,AGE_ONLY,E06000005,ALL,50_54,7776
96,AGE_ONLY,E06000005,ALL,55_59,7731
97,AGE_ONLY,E06000005,ALL,60+,28916
98,AGE_ONLY,E06000005,ALL,60_64,6785


In [14]:
import gc

# Commit and close Sqlite connection
conn.commit()
conn.close()

# Force garbage collection
_ = gc.collect(2)